# Bike Sharing Linear Regression Model

## Steps
1. Reading , understanding and Visualizing the data
2. Preparing the data for Modelling
    - Train - Test split
    - Rescaling
3. Training the Model 
4. Residual Analysis
5. Predictions and Evaluations on the test set

### Step 1: Reading , understanding and Visualizing the data

In [ ]:
# Import Needed Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import calendar


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
# Setting Plot size globally for better visualization
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
# Read the dataset
bike_sharing = pd.read_csv('day.csv')
bike_sharing.shape

In [ ]:
bike_sharing.info()

"""No Missing Values and no datatype conversions required"""

In [ ]:
# Drop Insignificant Columns
# instant is unique as it is a record index - it doesn't add any value
# casual and registered are already captured in cnt and the target column is cnt, hence dropping casual and registered
# dropping dteday as the date doesn't add significance because it is already being consumed in other significant forms like month,weekday,holiday and so on
insig_cols = ['instant','dteday','casual','registered']
bike_sharing.drop(insig_cols,axis=1,inplace=True)
bike_sharing.columns

In [ ]:
# Visualizing the data for linearity and multi collinearity
plt.figure()
sns.pairplot(bike_sharing)
plt.show()

"""At this point,temp and atemp may be multi collinear (+vely correlated) and is obviously explainable because temp is the actual temperature and atemp is feeling temperatue. """

In [ ]:
# Visualizing the data: Continuous Independent Variables
plt.figure(figsize=(12,5))
sns.pairplot(data=bike_sharing,x_vars=['temp', 'atemp', 'hum', 'windspeed'],y_vars='cnt')
plt.suptitle('Analysis of Numerical Variables against target variable',y=1.1)
plt.show()

"""There seems to be linear correlation between temp vs cnt and atemp vs cnt"""

In [ ]:
# Visualizing the data: Categorical Independent Variables
plt.figure(figsize=(20,10))
plt.subplot(2,4,1)
sns.boxplot(x='yr',y='cnt',data=bike_sharing)
plt.subplot(2,4,2)
sns.boxplot(x='season',y='cnt',data=bike_sharing)
plt.subplot(2,4,3)
sns.boxplot(x='mnth',y='cnt',data=bike_sharing)
plt.subplot(2,4,4)
sns.boxplot(x='holiday',y='cnt',data=bike_sharing)
plt.subplot(2,4,5)
sns.boxplot(x='weekday',y='cnt',data=bike_sharing)
plt.subplot(2,4,6)
sns.boxplot(x='workingday',y='cnt',data=bike_sharing)
plt.subplot(2,4,7)
sns.boxplot(x='weathersit',y='cnt',data=bike_sharing)
plt.suptitle('Analysis of Categorical Variables against target variable')
plt.show()


#### Inferences
- year seems to have very good influence of total rental bikes as the entire distribution for 2019 is higher than that of 2018 indicating a pattern
- season seems to have influence on number of people opting for total rental bikes thereby months also have influence. People tend to opt for more rental bikes in Summer and Fall 
- workingday & weekday doesn't seem to influence total rental bikes as the median and distribution is similar. 
- Weather situation seems to influence on total rental bikes where people opt for more rental bikes in clear or partly cloudy weather  
- People tend to opt for little more rental bikes during no holidays than on holidays. The influence could be little less appreciable attributing to the less difference in median

### Step 2:  Preparing the data for Modelling

#### Encoding
 - yes/no variables are already encoded with 1/0. No change needed
 - Certain Nominal Variables are represented as Ordinal variables like season, month, weekday, weathersit. Those has to be converted and dummy encoded

In [ ]:
## Listing Categorical columns and its unique values
column_values = {}
col_list = ['yr','season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
for row in col_list:
    column_values[row] = list(bike_sharing[row].value_counts().index)

print(column_values)

In [ ]:
## Plug in string values from data dict for Nominal Variables which are represented as Ordinal values in the dataset
yr_mappings = {0:'2018',1:'2019'}
season_mappings = {1:'spring', 2:'summer', 3:'fall', 4:'winter'}
weathersit_mappings = {1:'Clear',2:'Mist_Cloudy',3:'Light_Snow',4:'Heavy_Rain'}

bike_sharing['yr'] = bike_sharing[['yr']].apply(lambda x : x.map(yr_mappings))
bike_sharing['season'] = bike_sharing[['season']].apply(lambda x : x.map(season_mappings))
bike_sharing['weathersit'] = bike_sharing[['weathersit']].apply(lambda x : x.map(weathersit_mappings))
bike_sharing['mnth'] = bike_sharing['mnth'].apply(lambda x : calendar.month_abbr[x])
bike_sharing['weekday'] = bike_sharing['weekday'].apply(lambda x : calendar.day_abbr[x])
bike_sharing.head()

In [ ]:
## Dummy encoding
var_list_encoding = ['yr','season','mnth','weekday','weathersit']
dummy_encoded_values = pd.get_dummies(data=bike_sharing[var_list_encoding],drop_first=True)

# Add the new encoded cols to original dataframe and drop the source columns
bike_sharing = pd.concat([bike_sharing,dummy_encoded_values],axis=1)
bike_sharing.drop(var_list_encoding,axis=1,inplace=True)

#### Split test train dataset

In [ ]:
df_train, df_test = train_test_split(bike_sharing,train_size=0.7,random_state=100)
df_train.columns

#### Scaling the features using MinMaxScaler

In [ ]:
# Normalize the numerical columns other than categorical dummy cols
num_vars = ['temp','atemp','hum','windspeed','cnt']
scaler = MinMaxScaler()
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])


### Step 3:  Model Building

#### Dividing training set to X and y

In [ ]:
y_train = df_train.pop('cnt')
X_train = df_train

In [ ]:
# Create Linear Regression Model
lm = LinearRegression()
lm.fit(X_train,y_train)

# Running RFE with output number of values as 20
output_var_count = 20
rfe = RFE(lm,n_features_to_select=output_var_count)
rfe = rfe.fit(X_train, y_train)


In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
resulting_rfe_cols = X_train.columns[rfe.support_]
resulting_rfe_cols

In [ ]:
X_train.columns[~rfe.support_]

#### Building using stats model to get detailed statistics

In [ ]:
# Keeping only the columns from RFE
X_train_rfe = X_train[resulting_rfe_cols]

In [ ]:
# Add Constant
def add_constant(X_train):
    return sm.add_constant(X_train)

In [ ]:
## Build Model and return summary
def build_model(X_train,y_train):
    lm = sm.OLS(y_train,X_train).fit() # Fitting the model
    return lm.summary(),lm # Return summary

In [ ]:
# Compute VIF
def compute_vif(X_train):
    vif = pd.DataFrame()
    vif['Features'] = X_train.columns
    vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2) # Rounding to 2 decimal values
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif
    

In [ ]:
# Building Model with 20 params from RFE
build_model(add_constant(X_train_rfe),y_train)


In [ ]:
# Computing VIF for 15 variables from RFE
compute_vif(X_train_rfe)

##### Interpretations:
- RFE chosen 20 variables is able to explain 84% variance in the target variable (Adjusted R square is 84%)
- atemp has high p value and high VIF, Let's start by removing atemp

In [ ]:
# Remove atemp which has very high p value and high VIF
cols_to_be_removed = ['atemp']
X_train_rfe_m = X_train_rfe.drop(cols_to_be_removed,axis=1)

In [ ]:
build_model(add_constant(X_train_rfe_m),y_train)

In [ ]:
compute_vif(X_train_rfe_m)

##### Interpretations:
- Adjusted R square has remained the same indicating that atemp is a redundant variable
- As indicated by EDA, atemp has very high correlation with temp, removing atemp also brought down vif of temp
- holiday has high p value although it has low VIF

As a rule of thumb, remove the variable with high p value, let's remove holiday

In [ ]:
# Remove atemp which has very high p value and high VIF
# Remove holiday as it has high p value
cols_to_be_removed = ['atemp','holiday']
X_train_rfe_m = X_train_rfe.drop(cols_to_be_removed,axis=1)

In [ ]:
build_model(add_constant(X_train_rfe_m),y_train)

In [ ]:
compute_vif(X_train_rfe_m)

##### Interpretations:
- Adjusted R square has remained the same indicating that removed variables may not be good value add to the fitness of the model
- mnth_Feb has high p value > 0.05

let's remove mnth_Feb

In [ ]:
# Remove atemp which has very high p value and high VIF
# Remove holiday as it has high p value
# Remove mnth_Feb as it has high p value 
cols_to_be_removed = ['atemp','holiday','mnth_Feb']
X_train_rfe_m = X_train_rfe.drop(cols_to_be_removed,axis=1)

In [ ]:
build_model(add_constant(X_train_rfe_m),y_train)

In [ ]:
compute_vif(X_train_rfe_m)

##### Interpretations:
- Adjusted R square has remained the same indicating that removed variables may not be good value add to the fitness of the model
- hum has high vif

let's remove hum

In [ ]:
# Remove atemp which has very high p value and high VIF
# Remove holiday as it has high p value
# Remove mnth_Feb as it has high p value 
# Remove hum as it has high VIF
cols_to_be_removed = ['atemp','holiday','mnth_Feb','hum']
X_train_rfe_m = X_train_rfe.drop(cols_to_be_removed,axis=1)

In [ ]:
build_model(add_constant(X_train_rfe_m),y_train)

In [ ]:
compute_vif(X_train_rfe_m)

##### Interpretations:
- Adjusted R square has approx remained the same indicating that removed variables may not be good value add to the fitness of the model
- weekday_Mon has high p value

let's remove weekday_Mon

In [ ]:
# Remove atemp which has very high p value and high VIF
# Remove holiday as it has high p value
# Remove mnth_Feb as it has high p value 
# Remove hum as it has high VIF
# Remove weekday_Mon as it has high p value 
cols_to_be_removed = ['atemp','holiday','mnth_Feb','hum','weekday_Mon']
X_train_rfe_m = X_train_rfe.drop(cols_to_be_removed,axis=1)

In [ ]:
summary, final_model = build_model(add_constant(X_train_rfe_m),y_train)
summary

In [ ]:
compute_vif(X_train_rfe_m)

##### Interpretations:
- Adjusted R square has approx remained the same indicating that removed variables may not be good value add to the fitness of the model
- Since all the variables p values are less than 0.05, no feature elimination required based on p value
- Even though temp has slightly higher value of VIF, but removing temp will cause R square to drop by 5%. Since VIF is only slightly higher but temp being able to explain 5% more variance(Adjusted R square) in the data, let's keep temp and declare this as final model

### Step 4:  Residual Analysis

In [ ]:
y_train_pred = final_model.predict(add_constant(X_train_rfe_m))
y_train_pred

In [ ]:
# Plotting the residuals to confirm the Linear Regression Assumptions
res = y_train - y_train_pred
fig = plt.figure()
sns.distplot(res)
fig.suptitle('Error Terms', fontsize = 20)  
plt.xlabel('Errors', fontsize = 18) 
plt.show()

#### Assumptions verified:
- Residuals seems to have zero mean and follow normal distribution 

In [ ]:
plt.figure(figsize=(22,5))
plt.subplot(1,2,1)
sns.scatterplot(x=X_train_rfe_m['temp'],y=res)
plt.subplot(1,2,2)
sns.scatterplot(x=X_train_rfe_m['windspeed'],y=res)
plt.show()

#### Assumptions verified:
- Residuals doesn't seem to have any pattern and independent
- Residuals seems to have constant variance most of the time except for some outliers

### Step 5:  Predictions and Evaluations using the Final Model

#### Applying the Scaling on test sets


In [ ]:
# Normalize the numerical columns other than categorical dummy cols
num_vars = ['temp','atemp','hum','windspeed','cnt']
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.describe()

#### Splitting the test set into X and y


In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
# Removing columns which were removed during model building process
X_test_final = X_test[X_train_rfe_m.columns] 
X_test_final.columns

In [ ]:
y_test_pred = final_model.predict(add_constant(X_test_final))
y_test_pred

In [ ]:
# Rsquare of model from the test set
r_square_test = r2_score(y_test,y_test_pred)
r_square_test

#### Observations
- r_square of test set is just 3% less than that of training set. This implies model generalizes quite well